In [261]:
%reload_ext autoreload
%autoreload 2

In [262]:
import io
import pymongo
from IPython import display

from bokeh.io import output_notebook, show
from bokeh.palettes import all_palettes
from bokeh.layouts import gridplot

import climatools.lblnew.bestfit_params as bestfits
from climatools.cliradlw.dataset import load_output_file
from climatools.cliradlw.utils import mapband_new2old
import climatools.html.html as climahtml
from climatools.plot.plot import *


output_notebook()

Loading BokehJS ...

In [263]:
client = pymongo.MongoClient('localhost', 27017)

In [264]:
'''
Edit here to start with!
'''

GAS = 'co2'
BAND = 4
CONC = 400e-6
ATMPRO = 'mls'


def query_clirad():
    q = {'molecule': {GAS: CONC},
         'atmpro': ATMPRO,
         'band': [BAND],
         'commitnumber': 'a6676f4'}
    return {f'param.{n}': v for n, v in q.items()}



def query_lblnewbestfit():
    q = {'molecule': GAS,
         'conc': CONC,
         'atmpro': ATMPRO,
         'band': mapband_new2old()[BAND],
         'commitnumber': 'a22ab94'}
    return {f'param.{n}': v for n, v in q.items()}

In [265]:
DOC_CLIRAD = client.cliradnew.lw.find_one(query_clirad())
DS_FLUX = load_output_file(io.StringIO(DOC_CLIRAD['output_flux']))
DS_COOL = load_output_file(io.StringIO(DOC_CLIRAD['output_coolr']))

In [266]:
DOC_LBLNEW = client.lblnew.bestfit_lw.find_one(query_lblnewbestfit())
DS_FLUX_CRD = load_output_file(io.StringIO(DOC_LBLNEW['output_fluxg']))
DS_COOL_CRD = load_output_file(io.StringIO(DOC_LBLNEW['output_coolrg']))
DS_FLUX_WGT = load_output_file(io.StringIO(DOC_LBLNEW['output_wfluxg']))
DS_COOL_WGT = load_output_file(io.StringIO(DOC_LBLNEW['output_wcoolrg']))

In [267]:
def show_html(s):
    display.display(display.HTML(s))
    
def show_markdown(s):
    display.display(display.Markdown(s))

In [268]:
def script_print_param():
    docs = [DOC_CLIRAD, DOC_LBLNEW]
    params = [doc['param'] for doc in docs]
    
    param_bestfit = bestfits.kdist_params(molecule=GAS,
                                          band=mapband_new2old()[BAND])    
    params.append(param_bestfit)
    labels = ['clirad', 'lblnew', "clirad's k-tables' lblnew"]
    
    df = pd.DataFrame(columns=labels)
    
    for label, param in zip(labels, params):
        for n, v in param.items():
            df.loc[n, label] = str(v)
            
    df.index.name = 'Input parameter'
    df.columns.name = 'Run'
    
    display.display(df)
    show_markdown('*TABLE.* Input parameters of runs considered here.')

In [269]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)

    if 'g' in ds.dims:
        ds = ds.sum('g')
        
    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i']) 
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']



def pltdata_cool():
    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD',
             'srs': fmt_cool(DS_COOL),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(DS_COOL_CRD),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10',
             'srs': fmt_cool(DS_COOL_WGT.sel(igg=10)),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data



def plt_cool(pltdata=None):
    '''
    Plot cooling rate profiles on linear
    and log pressure axes.
    
    Parameters
    ----------
    pltdata: dict
        Plot data, including colour, line attributes, etc.
    everything: bokeh.models.layouts.Column
        Cooling rate profile plot.
    '''
    plin = plt_vert_profile_bokeh(pltdata=pltdata, 
                                  y_axis_type='linear',
                                  prange=(50, 1050))
    plin.plot_width, plin.plot_height = 400, 500
    
    plog = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='log',
                                  prange=(.01, 200)) 
    plog.plot_width, plog.plot_height = 400, 500
    
    everything = gridplot(plin, plog, ncols=2)
    return everything
    
    

def script_plt_cool():
    '''
    Script to plot the cooling rate profiles.
    '''
    data = pltdata_cool()
    p = plt_cool(pltdata=data)
    show(p)
    display.display(
        display.Markdown(f'*FIGURE.* Cooling rates. Summed over g.'))
    

In [270]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)

    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i'])
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df



def tbdata_flux():
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
        
    ilevels = [0, tropopauses[ATMPRO], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(DS_FLUX_CRD, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10',
            'df': fmt_flux(DS_FLUX_WGT.sel(igg=10), ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(DS_FLUX, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data



def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        show_markdown((f"*TABLE.* Flux. {d['label']}"))
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        show_markdown(("*TABLE.* Flux difference."
                       f"({d['label']}) - ({d0['label']})"))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        

        
def script_tb_flux():
    data = tbdata_flux()
    tb_flux(tbdata=data)

In [271]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [272]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [273]:
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    show_markdown('''
    commit a6676f4
    Adjust wgt in co2 band04
    wgt[4] -= .05
    wgt[5] -= .1
    ''')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    script_print_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cool()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()

In [274]:
script()

### CLIRAD-LW


    commit a6676f4
    Adjust wgt in co2 band04
    wgt[4] -= .05
    wgt[5] -= .1
    

### Table of Contents

# Input Parameters

Run,clirad,lblnew,clirad's k-tables' lblnew
Input parameter,,,
commitnumber,a6676f4,a22ab94,a06b618
molecule,{'co2': 0.0004},co2,co2
band,[4],3b,3b
atmpro,mls,mls,mls
tsfc,294,294,294
conc,NaN,0.0004,0.0004
dv,NaN,0.001,0.001
klin,NaN,0,0
ng_adju,NaN,"[0, 0]","[0, 0]"


...

*TABLE.* Flux difference.(CLIRAD) - (CRD)

# Cooling Rate by Layer

CRD            CLIRAD `coolr_bands`           
            layer     coolrg                layer     coolrg
pressure                                                    
0.000312        1  76.722160                    1  39.292440
0.000750        2  40.961180                    2  34.249738
0.001052        3  33.041424                    3  31.994849
0.001476        4  26.177150                    4  28.921894
0.002070        5  20.453586                    5  24.934229
0.002904        6  15.887011                    6  20.248772
0.004074        7  12.355441                    7  15.279224
0.005714        8   9.670386                    8  10.722995
0.008015        9   7.632119                    9   7.200701
0.011243       10   6.059189                   10   5.042465
0.015771       11   4.800884                   11   3.961394
0.022122       12   3.733240                   12   3.368927
0.031031       13   2.741506                   13   2.797625
0.043528       14   1.535104                   14   1.820151
0.061057       15   0.621270                   15   0.941904
0.085645       16   1.009105                   16   1.170861
0.120136       17   1.978014                   17   1.894187
0.168516       18   2.672370                   18   2.338524
0.236378       19   3.351253                   19   2.846526
0.331549       20   4.134225                   20   3.723491
0.465100       21   5.128998                   21   5.067308
0.652400       22   6.668315                   22   6.944673
0.915100       23   8.412267                   23   8.635709
1.283650       24   8.111907                   24   7.912534
1.800600       25   6.297593                   25   5.814562
2.525700       26   4.914451                   26   4.673450
3.542800       27   3.934145                   27   4.141310
4.969550       28   3.201959                   28   3.407321
6.970850       29   2.668698                   29   2.605599
9.778100       30   2.244095                   30   2.108119
13.715850      31   1.822880                   31   1.953571
19.239350      32   1.385437                   32   1.658963
26.987250      33   1.051152                   33   1.193750
37.855300      34   0.824001                   34   0.799794
53.100050      35   0.576221                   35   0.592731
73.887500      36   0.318069                   36   0.422377
97.662500      37   0.177026                   37   0.260479
121.437500     38   0.131797                   38   0.156201
145.212500     39   0.032605                   39   0.008916
168.987500     40  -0.100428                   40  -0.150828
192.762500     41  -0.096020                   41  -0.143729
216.537500     42   0.023673                   42  -0.005243
240.312500     43   0.075846                   43   0.057251
264.087500     44   0.091997                   44   0.074600
287.862500     45   0.098112                   45   0.081759
311.637500     46   0.099724                   46   0.084930
335.412500     47   0.098896                   47   0.089382
359.187500     48   0.096725                   48   0.090827
382.962500     49   0.093992                   49   0.091455
406.737500     50   0.091034                   50   0.091205
430.512500     51   0.087524                   51   0.089436
454.287500     52   0.082854                   52   0.084175
478.062500     53   0.076838                   53   0.077959
501.837500     54   0.070158                   54   0.069658
525.612500     55   0.063739                   55   0.061555
549.387500     56   0.058049                   56   0.053841
573.162500     57   0.053072                   57   0.046846
596.937500     58   0.048726                   58   0.040704
620.712500     59   0.044924                   59   0.035469
644.487500     60   0.041549                   60   0.031080
668.262500     61   0.038595                   61   0.027623
692.037500     62   0.036114                   62   0.023942
715.812500     63   0.034227                   63

# Fluxes by Layer

CRD                           CLIRAD                \
                        flug       fldg      fnetg       flug          fldg   
pressure    level                                                             
0.000000    1     -17.779314   0.000000 -17.779314 -17.396982  9.569022e-07   
0.000624    2     -17.779655   0.006016 -17.773639 -17.397346  3.272028e-03   
0.000876    3     -17.779811   0.007392 -17.772419 -17.397494  4.440906e-03   
0.001229    4     -17.780008   0.008970 -17.771038 -17.397689  5.972221e-03   
0.001723    5     -17.780256   0.010752 -17.769504 -17.397942  7.920695e-03   
0.002417    6     -17.780569   0.012747 -17.767822 -17.398273  1.030124e-02   
0.003391    7     -17.780969   0.014979 -17.765990 -17.398705  1.306872e-02   
0.004757    8     -17.781482   0.017490 -17.763992 -17.399274  1.610982e-02   
0.006672    9     -17.782148   0.020351 -17.761797 -17.400026  1.929571e-02   
0.009359    10    -17.783026   0.023658 -17.759368 -17.401033  2.259407e-02   
0.013128    11    -17.784201   0.027538 -17.756663 -17.402392  2.620471e-02   
0.018415    12    -17.785805   0.032149 -17.753656 -17.404263  3.055698e-02   
0.025830    13    -17.788054   0.037677 -17.750377 -17.406894  3.614789e-02   
0.036232    14    -17.791335   0.044337 -17.746999 -17.410699  4.340039e-02   
0.050823    15    -17.796754   0.052409 -17.744346 -17.416789  5.263615e-02   
0.071291    16    -17.806083   0.063244 -17.742839 -17.426848  6.497957e-02   
0.100000    17    -17.819859   0.080451 -17.739408 -17.441356  8.347015e-02   
0.140271    18    -17.837769   0.107797 -17.729971 -17.460025  1.111757e-01   
0.196760    19    -17.860581   0.148492 -17.712089 -17.483526  1.503262e-01   
0.275997    20    -17.888973   0.208339 -17.680634 -17.512371  2.058925e-01   
0.387100    21    -17.922659   0.296436 -17.626224 -17.545964  2.884946e-01   
0.543100    22    -17.958270   0.426827 -17.531443 -17.580375  4.165557e-01   
0.761700    23    -17.978681   0.619912 -17.358769 -17.598323  6.143532e-01   
1.068500    24    -17.941817   0.888772 -17.053045 -17.561110  8.910176e-01   
1.498800    25    -17.834572   1.195008 -16.639564 -17.457618  1.190885e+00   
2.102400    26    -17.688006   1.498725 -16.189282 -17.315640  1.464696e+00   
2.949000    27    -17.512420   1.815988 -15.696432 -17.144515  1.762299e+00   
4.136600    28    -17.309235   2.166257 -15.142978 -16.944400  2.144843e+00   
5.802500    29    -17.080558   2.569448 -14.511110 -16.717869  2.590775e+00   
8.139200    30    -16.822858   3.050440 -13.772418 -16.457836  3.052042e+00   
11.417000   31    -16.527399   3.626315 -12.901084 -16.154238  3.567068e+00   
16.014700   32    -16.200522   4.292234 -11.908288 -15.813597  4.290510e+00   
22.464000   33    -15.884825   5.034963 -10.849862 -15.482851  5.227286e+00   
31.510500   34    -15.603034   5.879610  -9.723424 -15.181395  6.205211e+00   
44.200100   35    -15.321586   6.836779  -8.484808 -14.872308  7.098477e+00   
62.000000   36    -15.069403   7.799571  -7.269832 -14.584016  8.060102e+00   
85.775000   37    -14.997598   8.623551  -6.374047 -14.479897  9.145656e+00   
109.550000  38    -15.099057   9.223570  -5.875487 -14.552134  9.951560e+00   
133.325000  39    -15.263683   9.759378  -5.504305 -14.687175  1.052656e+01   
157.100000  40    -15.604738  10.192260  -5.412478 -15.014605  1.087910e+01   
180.875000  41    -16.352852  10.657536  -5.695316 -15.785206  1.122488e+01   
204.650000  42    -17.440965  11.475226  -5.965739 -16.919641  1.195449e+01   
228.425000  43    -18.588967  12.689898  -5.899069 -18.115668  1.313575e+01   
252.200000  44    -19.724622  14.039158  -5.685464 -19.292179  1.447351e+01   
275.975000  45    -20.842292  15.415922  -5.426370 -20.445948  1.583740e+01   
299.750000  46    -21.940873  16.790817  -5.150056 -21.576442  1.719818e+01   
323.525000  47    -23.019646  18.150446  -4.869201 -22.684150  1.854510e+01   
347.300000  48    -24.077740  19.487061  -4.590679 -23.767671  1.988038e+01

In [275]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')